In [ ]:
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from stagpy.stagyydata import StagyyData
from stagpy import field as sp_field
from stagpy import error

# --- USER INPUT ---
target_time_Gyr = 1.0  # The time you want to find (in Gyr)
field_to_plot = "eta"   # Choose "T", "eta", or "basalt"

# --- CONFIGURATION ---
# Adjust these limits to match your model's physical range
field_limits = {
    "T": {"vmin": 300, "vmax": 4000},
    "basalt": {"vmin": 0.0, "vmax": 1.0},
    "eta": {"vmin": 1e18, "vmax": 1e25}
}

field_labels = {
    "T": "Temperature",
    "eta": "Viscosity",
    "basalt": "Basalt Fraction"
}

# Data path
data_path = Path("/media/aritro/f522493b-003a-404d-a839-3e0925c674b6/Aritro/StagYY/runs/venus_01/archive/")
sdat = StagyyData(data_path)

SEC_PER_GYR = 1e9 * 365.25 * 24 * 3600

# 1. SCAN SNAPSHOTS FOR THE CLOSEST TIME
print(f"Searching for snapshot closest to {target_time_Gyr} Gyr...")
snap_indices = []
snap_times_seconds = []

for snap in sdat.snaps:
    try:
        t = snap.timeinfo["t"]
        snap_indices.append(snap.isnap)
        snap_times_seconds.append(t)
    except:
        continue

snap_times_seconds = np.array(snap_times_seconds)
snap_indices = np.array(snap_indices)

# Find the best match
target_seconds = target_time_Gyr * SEC_PER_GYR
idx = np.abs(snap_times_seconds - target_seconds).argmin()
snap_number = int(snap_indices[idx])
actual_time_Gyr = snap_times_seconds[idx] / SEC_PER_GYR

print(f"Target Time: {target_time_Gyr} Gyr")
print(f"Closest Found: Snapshot {snap_number} at {actual_time_Gyr:.4f} Gyr")

# 2. GENERATE THE PLOT
try:
    snapshot = sdat.snaps[snap_number]
    lims = field_limits.get(field_to_plot)
    
    # Handle Logarithmic scaling for eta
    if field_to_plot == "eta":
        norm = colors.LogNorm(vmin=lims["vmin"], vmax=lims["vmax"])
        fig, ax, mesh, cbar = sp_field.plot_scalar(snapshot, field_to_plot, norm=norm)
    else:
        fig, ax, mesh, cbar = sp_field.plot_scalar(snapshot, field_to_plot, 
                                                 vmin=lims["vmin"], vmax=lims["vmax"])
    
    # Metadata and Labels
    unit = snapshot.fields[field_to_plot].meta.dim
    display_name = field_labels.get(field_to_plot, field_to_plot)
    cbar.set_label(f"{display_name} [{unit}]")
    
    # Add Timestamp to plot
    ax.text(0.5, 0.5, f"{actual_time_Gyr:.3f} Gyr", 
            transform=ax.transAxes, ha="center", va="center", 
            fontsize=20, color="black", 
            bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))
    
    # Formatting (Wider figure to prevent truncation)
    fig.set_size_inches(10, 6)
    plt.tight_layout()
    
    # Save the file
    save_name = f"single_frame_{field_to_plot}_{actual_time_Gyr:.3f}Gyr.png"
    fig.savefig(save_name, dpi=400)
    plt.close(fig)
    
    print(f"Successfully saved: {save_name}")

except Exception as e:
    print(f"An error occurred during plotting: {e}")